# Исследование надежности заемщиков


Заказчик — кредитный отдел банка. 

Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


Описание данных
children — количество детей в семье
days_employed — общий трудовой стаж в днях
dob_years — возраст клиента в годах
education — уровень образования клиента
education_id — идентификатор уровня образования
family_status — семейное положение
family_status_id — идентификатор семейного положения
gender — пол клиента
income_type — тип занятости
debt — имел ли задолженность по возврату кредитов
total_income — ежемесячный доход
purpose — цель получения кредита


## Откройте таблицу и изучите общую информацию о данных

In [1]:
import pandas as pd

data = pd.read_csv('/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

Количество пропущенных значений для каждого столбца.

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаете на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

Удаляем аномалии в столбце days_emploeyd, путем применения функции abs напрямую к столбцу.

In [6]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения.

**Задание 8. Выведите перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**Задание 9. В столбце `children` есть два аномальных значения. Удалите строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Задание 10. Ещё раз выведите перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

**Задание 11. Заполните пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Задание 12. Убедитесь, что все пропуски заполнены. Проверьте себя и ещё раз выведите количество пропущенных значений для каждого столбца с помощью двух методов.**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Задание 13. Замените вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Задание 14. Обработайте неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру. Проверьте остальные столбцы.**

In [14]:
data['education'] = data['education'].str.lower()

**Задание 15. Выведите на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалите их.**

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

**Задание 16. На основании диапазонов, указанных ниже, создайте в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'

        elif 30001 <= income <= 50000:
            return 'D'

        elif 50001 <= income <= 200000:
            return 'C'

        elif 200001 <= income <= 1000000:
            return 'B'

        elif income >= 1000001:
            return 'A'
            
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Перечень уникальных целей взятия кредита из столбца `purpose`.

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Задание 18. Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используйте собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучите данные в столбце `purpose` и определите, какие подстроки помогут вам правильно определить категорию.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)
print(data)

       children  days_employed  dob_years education  education_id  \
0             1    8437.673028         42    высшее             0   
1             1    4024.803754         36   среднее             1   
2             0    5623.422610         33   среднее             1   
3             3    4124.747207         32   среднее             1   
4             0  340266.072047         53   среднее             1   
...         ...            ...        ...       ...           ...   
21520         1    4529.316663         43   среднее             1   
21521         0  343937.404131         67   среднее             1   
21522         1    2113.346888         38   среднее             1   
21523         3    3112.481705         38   среднее             1   
21524         2    1984.507589         40   среднее             1   

          family_status  family_status_id gender income_type  debt  \
0       женат / замужем                 0      F   сотрудник     0   
1       женат / замужем        

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [66]:
debts = data.groupby('children')['debt'].sum().sort_values(ascending=False)
all_borrowers = []
childrens = []
debts_borrowers = []
debts_part = []

for ch in range(6):
    borrowers = len(data[data['children'] == ch])
    all_borrowers.append(borrowers)
    childrens.append(ch)
    debts_borrowers.append(debts[ch])
    debts_part.append(round(debts[ch] / borrowers * 100, 2))
    
borrowers_ch = pd.DataFrame({
    'childrens': childrens,
    'all_borrowers': all_borrowers,
    'debts_borrowers': debts_borrowers,
    'debts_part': debts_part
})

print(borrowers_ch)

   childrens  all_borrowers  debts_borrowers  debts_part
0          0          14091             1063        7.54
1          1           4808              444        9.23
2          2           2052              194        9.45
3          3            330               27        8.18
4          4             41                4        9.76
5          5              9                0        0.00


**Вывод:** кардинальная разница видна только в случае заемщика с пятью детьми в семье, при этом стоит учитывать, что выборка маленькая - таких заемщиков всего 9 человек. При взгляде на абсолютные величины видно, что чем больше детей, тем меньше заемщиков-должников. Стоит обратить внимание на относительные значения - становится понятно, что разница небольшая. Таким образом, можно сказать, что зависимость между количеством детей и возвратом кредита в срок отсутствует.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [67]:
families_debt = data.groupby('family_status')['debt'].sum().sort_values(ascending=False)

all_borrowers = []
statuses = ['женат / замужем', 'гражданский брак', 'Не женат / не замужем', 'в разводе', 'вдовец / вдова']
debts_borrowers = []
debts_part = []

for ind, fs in enumerate(statuses):
    borrowers = len(data[data['family_status'] == fs])
    all_borrowers.append(borrowers)
    debts_borrowers.append(families_debt[fs])
    debts_part.append(round(families_debt[ind] / borrowers * 100, 2))
    
borrowers_fs = pd.DataFrame({
    'family_status': statuses,
    'all_borrowers': all_borrowers,
    'debts_borrowers': debts_borrowers,
    'debts_part': debts_part
})

print(borrowers_fs)

           family_status  all_borrowers  debts_borrowers  debts_part
0        женат / замужем          12261              927        7.56
1       гражданский брак           4134              385        9.31
2  Не женат / не замужем           2796              273        9.76
3              в разводе           1189               84        7.06
4         вдовец / вдова            951               63        6.62


**Вывод:** зависимость между семейным положением и возвратом кредита в срок наблюдается. Чаще кредит в срок возвращают вдовы / вдовцы, реже - не женатые / не замужем.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [70]:
categories_debt = data.groupby('total_income_category')['debt'].sum().sort_values(ascending=False)
categories = ['C', 'B', 'D', 'A', 'E']
all_borrowers = []
debts_borrowers = []
debts_part = []

for ind, cat in enumerate(categories):
    borrowers = len(data[data['total_income_category'] == cat])
    all_borrowers.append(borrowers)
    debts_borrowers.append(categories_debt[cat])
    debts_part.append(round(categories_debt[ind] / borrowers * 100, 2))
    
borrowers_cat = pd.DataFrame({
    'category': categories,
    'all_borrowers': all_borrowers,
    'debts_borrowers': debts_borrowers,
    'debts_part': debts_part
})

print(borrowers_cat)

  category  all_borrowers  debts_borrowers  debts_part
0        C          15921             1353        8.50
1        B           5014              354        7.06
2        D            349               21        6.02
3        A             25                2        8.00
4        E             22                2        9.09


**Вывод:** разный уровень дохода действительно влияет на возврат кредита в срок. Люди, категория доходов которых равна 'E' (0 - 30000) реже всех возвращают кредит в срок. При этом чаще всего кредит в срок возвращают заемщики с категорией доходов 'D' (30001–50000) Вместе с тем, важно обратить внимание, что выборка крайне несбалансировнная - заемщиков с категорией доходов 'C' - 15921, а с категорией 'B' уже всего лишь 5014.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [71]:
purposes_debt = data.groupby('purpose_category')['debt'].sum().sort_values(ascending=False)
purposes = ['операции с недвижимостью', 'операции с автомобилем', 'получение образования', 'проведение свадьбы']
all_borrowers = []
debts_borrowers = []
debts_part = []

for ind, purp in enumerate(purposes):
    borrowers = len(data[data['purpose_category'] == purp])
    all_borrowers.append(borrowers)
    debts_borrowers.append(purposes_debt[purp])
    debts_part.append(round(purposes_debt[ind] / borrowers * 100, 2))
    
borrowers_purp = pd.DataFrame({
    'purposes': purposes,
    'all_borrowers': all_borrowers,
    'debts_borrowers': debts_borrowers,
    'debts_part': debts_part
})

print(borrowers_purp)

                   purposes  all_borrowers  debts_borrowers  debts_part
0  операции с недвижимостью          10751              780        7.26
1    операции с автомобилем           4279              400        9.35
2     получение образования           3988              369        9.25
3        проведение свадьбы           2313              183        7.91


**Вывод:** чаще всего кредит в срок возвращают люди, которые брали его с целью проведения свадьбы. Реже всего - заемщики, проводящие операции с автомобилем.

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* можно выделить две причины появления пропусков: зависящие от человека, и зависящие от программного обеспечения. Например, в таблице есть строки с пустыми значениям 'total_income'. Возможно, работник банка при оформлении кредита не ввел эти данные в заявку (человеческий фактор). Или, например, строки, где значение 'days_employed' (трудовой стаж в днях) идет с минусом в начале. Скорее всего, произошла какая-то ошибка при выгрузке данных.

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* в случае количественных переменных, лучше использовать медианное значение, потому что в данных могут быть слишком большие или слишком маленькие значения, которые по итогу сместят среднее значение.

### Шаг 4: общий вывод.

В ходе работы был проведен анализ таблицы с информацией о заемщиках, были удалены пропуски и обработаны аномальные значения. Также была воспроизведена категория данных - созданы два новых столбца с категориями доходов и категориями целей взятия кредитов. Какие выводы можно сделать: во первых, точно нет никакой зависимости между количеством детей и возвратом кредита в срок. Также важно уточнить, что заемщик с пятью детьми не является надежным, из-за нулевого процента должников (учитываем, что таких заемщиков всего девять). Наблюдается связь между семейным положением и возвратом кредита вовремя: чащего всего в срок возвращают вдовы / вдовцы, реже всего - не женатые / не замужем. Разный уровень дохода действительно влияет на возврат кредита в срок. Так, люди с категорией доходов 'E' реже всего возвращают кредит в срок, в то время как люди с категорией доходов 'D' - наоборот, чаще. Важно помнить про цели взятия кредита: чаще вовремя возвращают кредит, взятый с целью проведения свадьбы, реже - проводящие операции с автомобилем.